In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import gc

In [2]:
order_products = pd.concat([pd.read_csv('../data/order_products__prior.csv'),
                            pd.read_csv('../data/order_products__train.csv')])
products = pd.read_csv('../data/products.csv')
departments = pd.read_csv('../data/departments.csv')

In [3]:
df = pd.merge(order_products, products, on='product_id')

#cleanup
del order_products
del products
del departments
gc.collect()

df.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,2,33120,1,1,Organic Egg Whites,86,16
1,26,33120,5,0,Organic Egg Whites,86,16
2,120,33120,13,0,Organic Egg Whites,86,16
3,327,33120,5,1,Organic Egg Whites,86,16
4,390,33120,28,1,Organic Egg Whites,86,16


In [4]:
# Indicator variable for presence of product category in order
df['values'] = 1
df1 = df.pivot_table(values='values', index='order_id', columns='department_id').fillna(0.0).reset_index()
df1.head()

department_id,order_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
0,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,5,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0


In [5]:
# count of product category in order
df['department_id_count'] = df.groupby(['order_id', 'department_id'])['order_id'].transform('count')
df2 = df.pivot_table(values='department_id_count', index='order_id', columns='department_id').fillna(0.0).reset_index()
df2.head()

department_id,order_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
0,1,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
3,4,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0
4,5,0.0,0.0,0.0,7.0,0.0,1.0,1.0,0.0,2.0,0.0,1.0,1.0,2.0,0.0,0.0,3.0,3.0,0.0,4.0,1.0,0.0


In [6]:
# cleanup
del df
gc.collect()

# final cleaned dataset
orders = pd.read_csv('../data/orders.csv')
df_clean = pd.merge(df1, df2, on='order_id')
df_clean = pd.merge(df_clean, orders[['order_id', 'order_dow', 'order_hour_of_day']], on='order_id')

# renaming columns
df_clean = df_clean.rename(columns={**{f"{i}_x":f"category_{i}" for i in range(1, 22)},
                                    **{f"{i}_y":f"category_{i}_count" for i in range(1, 22)}
                                   }
                          )

df_clean.head()

,order_id,category_1,category_2,category_3,category_4,category_5,category_6,category_7,category_8,category_9,category_10,category_11,category_12,category_13,category_14,category_15,category_16,category_17,category_18,category_19,category_20,category_21,category_1_count,category_2_count,category_3_count,category_4_count,category_5_count,category_6_count,category_7_count,category_8_count,category_9_count,category_10_count,category_11_count,category_12_count,category_13_count,category_14_count,category_15_count,category_16_count,category_17_count,category_18_count,category_19_count,category_20_count,category_21_count,order_dow,order_hour_of_day
0,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,4,10
1,2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5,9
2,3,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,5,17
3,4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,1,9
4,5,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,7.0,0.0,1.0,1.0,0.0,2.0,0.0,1.0,1.0,2.0,0.0,0.0,3.0,3.0,0.0,4.0,1.0,0.0,6,16


In [9]:
df_clean.to_csv('../data/dataset_clean.csv', index=False)